In [1]:
import keras as keras
import numpy as np
import pandas as pd
import matplotlib as plt

% matplotlib inline

/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/amit/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstr

In [2]:
DATA_DIR = "../../data/movie-lens/"
df = pd.read_csv(DATA_DIR + "ratings.csv")

In [3]:
df.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [4]:
df.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [5]:
df["user_cat"] = df["userId"].astype("category").cat.codes
df["movie_cat"] = df["movieId"].astype("category").cat.codes

n_user = len(df["userId"].astype("category").cat.categories)
n_movie = len(df["movieId"].astype("category").cat.categories)

In [6]:
df.head()

,userId,movieId,rating,timestamp,user_cat,movie_cat
0,1,31,2.5,1260759144,0,30
1,1,1029,3.0,1260759179,0,833
2,1,1061,3.0,1260759182,0,859
3,1,1129,2.0,1260759185,0,906
4,1,1172,4.0,1260759205,0,931


In [7]:
df.describe()

,userId,movieId,rating,timestamp,user_cat,movie_cat
count,100004.000000,100004.000000,100004.000000,1.000040e+05,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608,1.129639e+09,346.011310,2691.887445
std,195.163838,26369.198969,1.058064,1.916858e+08,195.163838,2375.936209
min,1.000000,1.000000,0.500000,7.896520e+08,0.000000,0.000000
25%,182.000000,1028.000000,3.000000,9.658478e+08,181.000000,832.000000
50%,367.000000,2406.500000,4.000000,1.110422e+09,366.000000,1916.500000
75%,520.000000,5418.000000,4.000000,1.296192e+09,519.000000,4121.000000
max,671.000000,163949.000000,5.000000,1.476641e+09,670.000000,9065.000000


In [8]:
# build model
from keras.models import *
from keras.layers import *

USER_EMBEDDING_DIM = 20
MOVIE_EMBEDDING_DIM = 40

movie_input = Input(shape=(1,))
movie_embed = Flatten()(Embedding(n_movie, MOVIE_EMBEDDING_DIM)(movie_input))
movie_embed = Dropout(0.4)(movie_embed)

user_input = Input(shape=(1,))
user_embed = Flatten()(Embedding(n_user, USER_EMBEDDING_DIM)(user_input))
user_embed = Dropout(0.3)(user_embed)

x = concatenate([movie_embed, user_embed])
x = Dense(50)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(20)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

out = Dense(1)(x)

model = Model(inputs=[movie_input, user_input], outputs=out)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 40)        362640      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 20)        13420       input_2[0][0]                    
__________________________________________________________________________________________________
flatten_1 

In [9]:
model.compile(optimizer='sgd', loss='mse')

In [10]:
model.fit([df["movie_cat"], df["user_cat"]], df["rating"], validation_split=0.12, epochs=20)

Train on 88003 samples, validate on 12001 samples
Epoch 1/20
88003/88003 [==============================] - 18s 210us/step - loss: 1.2203 - val_loss: 1.0579
Epoch 2/20
88003/88003 [==============================] - 17s 188us/step - loss: 0.9899 - val_loss: 0.9925
Epoch 3/20
88003/88003 [==============================] - 18s 200us/step - loss: 0.9385 - val_loss: 0.9626
Epoch 4/20
88003/88003 [==============================] - 17s 189us/step - loss: 0.8996 - val_loss: 0.9333
Epoch 5/20
88003/88003 [==============================] - 17s 189us/step - loss: 0.8726 - val_loss: 0.9300
Epoch 6/20
88003/88003 [==============================] - 18s 203us/step - loss: 0.8531 - val_loss: 0.9259
Epoch 7/20
88003/88003 [==============================] - 18s 199us/step - loss: 0.8395 - val_loss: 0.9044
Epoch 8/20
88003/88003 [==============================] - 18s 203us/step - loss: 0.8280 - val_loss: 0.8956
Epoch 9/20
88003/88003 [==============================] - 18s 202us/step - loss: 0.8174 - val_

In [56]:
model.fit([df["movie_cat"], df["user_cat"]], df["rating"], validation_split=0.12, epochs=10)

Train on 88003 samples, validate on 12001 samples
Epoch 1/10
88003/88003 [==============================] - 18s 201us/step - loss: 0.7745 - val_loss: 0.8793
Epoch 2/10
88003/88003 [==============================] - 18s 202us/step - loss: 0.7733 - val_loss: 0.8815
Epoch 3/10
88003/88003 [==============================] - 18s 203us/step - loss: 0.7698 - val_loss: 0.8812
Epoch 4/10
88003/88003 [==============================] - 18s 204us/step - loss: 0.7686 - val_loss: 0.8812
Epoch 5/10
88003/88003 [==============================] - 18s 201us/step - loss: 0.7652 - val_loss: 0.8774
Epoch 6/10
88003/88003 [==============================] - 18s 202us/step - loss: 0.7629 - val_loss: 0.8803
Epoch 7/10
88003/88003 [==============================] - 19s 212us/step - loss: 0.7660 - val_loss: 0.8825
Epoch 8/10
88003/88003 [==============================] - 19s 218us/step - loss: 0.7619 - val_loss: 0.8808
Epoch 9/10
88003/88003 [==============================] - 19s 220us/step - loss: 0.7563 - val_